# TAXII 2.0 Tutorial

Connect to the UberTAXII server, list API roots, and pull some STIX data.

In [ ]:
# A "Server" object pulls general information about the server, and gives you a list of API roots.
# Connect to it by providing the URL, username, and password.

from taxii2client import Server
server = Server('https://ubertaxii.com/taxii/', user='ubertaxii', password='certain_players_want_a_pony')

In [ ]:
# Print some basic information, available without making additional requests.

print("Title:\t\t" + server.title)
print("Description:\t" + server.description)

In [ ]:
# Iterate through the API roots. Note that addtional requests are made automatically to get the title and description.

for api_root in server.api_roots:
    print(api_root.title)
    print(api_root.description)
    print(api_root.url)
    print("\n")

In [ ]:
# Get the list of collections for an API root (using MITRE's)

mitre_api_root = server.api_roots[3]
for collection in mitre_api_root.collections:
    print(collection.title)

# STIX 2.0 Tutorial

Starting with our reference to a TAXII server, do some stuff with STIX data

In [ ]:
from stix2 import TAXIICollectionSource, Filter

taxii_source = TAXIICollectionSource(mitre_api_root.collections[0])

In [ ]:
results = taxii_source.query(Filter("type", "=", "intrusion-set"))
print("Got {} results".format(len(results)))

In [ ]:
apt_xxx = results[36]
print(apt_xxx.name) # Note that it's a python-stix2 object already, no need to parse the JSON

In [ ]:
# What malware and tools does that intrusion set use?

# Find relationships where the source is that intrusion set
all_relationships = taxii_source.query(Filter("type", "=", "relationship"))
relationships = [r for r in all_relationships if r.source_ref == apt_xxx.id]
print("Found {} relationships".format(len(relationships)))

In [ ]:
for rel in relationships:
    target = taxii_source.get(rel.target_ref)
    print("{} {} {} ({})".format(apt_xxx.name, rel.relationship_type, target.name, target.type))

# Gotcha! Note that this does not optimize the number of calls.
# You still need to know when a TAXII request is going to be made and when the result is available locally.
# Also, watch for n+1 queries (like here) and try to optimize if possible

## Make it Easy: Environments
Use environments to add default sources, sinks, etc.

In [ ]:
from stix2 import Environment, MemoryStore

taxiisource = TAXIICollectionSource(mitre_api_root.collections[0])
env = Environment(source=taxiisource)

In [ ]:
env.get(apt_xxx.id).name

In [ ]:
# Try the other direction
rels = env.query(Filter("source_ref", "=", apt_xxx.id))
print(len(rels))

# Note the delay: not all filters happen server-side,
# it sometimes transparently just pulls all data and filters at the client

In [ ]:
malware = env.get(rels[1].target_ref)
malware.name

## Sharing back: Creating STIX

You can also use the enviornment to set defaults so you don't have to duplicate adding standard markings, etc.

In [ ]:
from stix2 import Indicator, Relationship, ObjectFactory, CompositeDataSource

factory = ObjectFactory(created_by_ref="identity--311b2d2d-f010-5473-83ec-1edf84858f4c")
source = CompositeDataSource()
memorystore = MemoryStore()
source.add_data_sources([memorystore.source, taxiisource])
env2 = Environment(factory=factory, source=source, sink=memorystore)

ind = factory.create(Indicator,
                     labels=["malicious-activity"],
                     pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")
rel = factory.create(Relationship, source_ref=ind.id, target_ref=malware.id, relationship_type="indicates")
print(rel)

In [ ]:
# Push to our memory data source
env2.add(ind)
env2.add(rel)

In [ ]:
# Is it still there?
print(env2.get(rel.id))

In [ ]:
# setup writable API root
from stix2 import TAXIICollectionSink

scratch_api_root = server.api_roots[4]
for collection in scratch_api_root.collections:
    print(collection.title)
taxii_sink = TAXIICollectionSink(scratch_api_root.collections[0])
env3 = Environment(factory=factory, source=source, sink=taxii_sink)

In [ ]:
# Push to our memory data source
env3.add(ind)
env3.add(rel)

In [ ]:
# Is it still there?
print(env3.get(rel.id))

# NOTE: This is hitting the TAXII collection we just wrote to...